In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
  import numpy as np
  import pandas as pd
  from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
  from tensorflow.keras.preprocessing import image
  import os


  # Load the ResNet50 model pretrained on ImageNet
  model = ResNet50(weights='imagenet', include_top=False)

  # Path to the Kaggle Dogs vs. Cats dataset
  dataset_path = '/content/drive/MyDrive/train'

  # Function to extract features from an image
  def extract_features(img_path):
      img = image.load_img(img_path, target_size=(224, 224))  # Resizing the image to match the input size of ResNet50
      img = image.img_to_array(img)  # Convert image to numpy array
      img = np.expand_dims(img, axis=0)  # Add an extra dimension to represent batch size
      img = preprocess_input(img)  # Preprocess the image (e.g., normalization)

      features = model.predict(img)  # Extract features using ResNet50
      features = features.flatten()  # Flatten the feature tensor to a 1D vector

      return features

  # List of image paths
  image_paths = []
  labels = []

  # Traverse the dataset directory and collect image paths and labels
  for root, dirs, files in os.walk(dataset_path):
      for file in files:
          if file.endswith('.jpg'):
              image_paths.append(os.path.join(root, file))
              labels.append(file.split('.')[0])

  # Extract features for each image and store them in a list
  features_list = []
  for img_path in image_paths:
      features = extract_features(img_path)
      features_list.append(features)

  # Convert the list of feature vectors and labels to a pandas DataFrame
  df = pd.DataFrame(features_list)
  df['label'] = labels

  # Save the DataFrame to a CSV file
  df.to_csv('image_features.csv', index=False)


Found GPU at: /device:GPU:0
1/1 [==============================] - 0s 22ms/step


In [3]:
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))

with tf.device('/device:GPU:0'):
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.svm import SVC
  from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

  # Load the extracted features from the CSV file
  df = pd.read_csv('image_features.csv')

  # Split the dataset into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(
      df.drop('label', axis=1), df['label'], test_size=0.3, random_state=42
  )

  # Initialize and train the SVM model with hyperparameters
  svm_model = SVC(C=1.0, kernel='rbf', gamma='scale', random_state=42)
  svm_model.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = svm_model.predict(X_test)

  # Print classification report
  print("Classification Report:")
  print(classification_report(y_test, y_pred))

  # Print accuracy score
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy:", accuracy)

  # Print confusion matrix
  confusion_mat = confusion_matrix(y_test, y_pred)
  print("Confusion Matrix:")
  print(confusion_mat)


Found GPU at: /device:GPU:0
Classification Report:
              precision    recall  f1-score   support

         cat       0.99      0.96      0.98       150
         dog       0.96      0.99      0.98       151

    accuracy                           0.98       301
   macro avg       0.98      0.98      0.98       301
weighted avg       0.98      0.98      0.98       301

Accuracy: 0.9767441860465116
Confusion Matrix:
[[144   6]
 [  1 150]]
